In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bigquery-geotab-intersection-congestion/sample_submission.csv
/kaggle/input/bigquery-geotab-intersection-congestion/submission_metric_map
/kaggle/input/bigquery-geotab-intersection-congestion/submission_metric_map.json
/kaggle/input/bigquery-geotab-intersection-congestion/train.csv
/kaggle/input/bigquery-geotab-intersection-congestion/test.csv
/kaggle/input/bigquery-geotab-intersection-congestion/BigQuery-Dataset-Access.md


In [2]:
train = pd.read_csv("/kaggle/input/bigquery-geotab-intersection-congestion/train.csv")
sample = pd.read_csv("/kaggle/input/bigquery-geotab-intersection-congestion/sample_submission.csv")
test = pd.read_csv("/kaggle/input/bigquery-geotab-intersection-congestion/test.csv")

In [3]:
test_norowid = test.drop("RowId",axis=1)

In [4]:
train = train.drop(train.loc[train['EntryStreetName'].isnull()].index)
train = train.drop(train.loc[train['ExitStreetName'].isnull()].index)

In [5]:
map_head = {'N':0,'NE':1,'E':2,'SE':3,'S':4,'SW':5,'W':6,'NW':7}
col_head = ["EntryHeading","ExitHeading"]

train_enc = train

train_enc[col_head] = train[col_head].applymap(map_head.get)

train_enc[col_head]

test[col_head] = test[col_head].applymap(map_head.get)

test[col_head]

,EntryHeading,ExitHeading
0,1,1
1,5,3
2,5,5
3,1,1
4,5,5
...,...,...
1921352,4,6
1921353,4,4
1921354,4,6
1921355,4,4


In [6]:
address_encoding = {
    "Street": 0,
     "St": 0,
     "Avenue": 1,
     "Ave": 1,
     "Boulevard": 2,
     "Road": 3,
     "Drive": 4,
     "Lane": 5,
     "Tunnel": 6,
     "Highway": 7,
     "Way": 8,
     "Parkway": 9,
     "Parking": 10,
     "Oval": 11,
     "Square": 12,
     "Place": 13,
     "Bridge": 14
}

def encode(x):
    if pd.isna(x):
        return None
    for road in address_encoding:
        if (road in x):
            return address_encoding[road]
    return None

In [7]:
train["EntryAdressEncoded"] = train['EntryStreetName'].apply(encode)
train['ExitAddressEncoded'] = train['ExitStreetName'].apply(encode)
test["EntryAdressEncoded"] = test['EntryStreetName'].apply(encode)
test['ExitAddressEncoded'] = test['ExitStreetName'].apply(encode)

In [8]:
train['CoordEncoded'] = train['Latitude'].astype(str) + train['Longitude'].astype(str)
train['CoordEncoded'] = train.groupby(['CoordEncoded']).ngroup()

test['CoordEncoded'] = test['Latitude'].astype(str) + test['Longitude'].astype(str)
test['CoordEncoded'] = test.groupby(['CoordEncoded']).ngroup()

In [9]:
train['CoordEncoded'].value_counts()

361     2437
368     2364
342     2206
352     2124
1605    2065
        ... 
2347       1
3367       1
3024       1
3193       1
952        1
Name: CoordEncoded, Length: 4801, dtype: int64

In [10]:
(train.loc[train["IntersectionId"] == 4])["Latitude"].value_counts()

33.817665    1545
42.339351     264
39.937021     240
41.875622      44
Name: Latitude, dtype: int64

In [11]:
(round(train["Latitude"], 6) == 33.817665).value_counts()
# train["Latitude"].head()

False    840907
True       1545
Name: Latitude, dtype: int64

In [12]:
train = train_enc

In [13]:
X=train[['CoordEncoded','EntryHeading','ExitHeading', 'Hour','Weekend']]
Y=train[['TotalTimeStopped_p20','DistanceToFirstStop_p20','TotalTimeStopped_p50','DistanceToFirstStop_p50', 'TotalTimeStopped_p80','DistanceToFirstStop_p80']]

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing


X_train, X_validate, Y_train, Y_validate = train_test_split(X, Y, test_size = 0.2, random_state = 1)

In [15]:
model = LinearRegression()
model.fit(X_train,Y_train)
Y_pred = model.predict(X_validate)

model.score(X_train,Y_train)

0.007866807653040597

In [16]:
# import matplotlib.pyplot as plt

# Y_hat = model.predict(X_train)
# plt.scatter(Y_train,Y_hat,alpha = 0.2)
# plt.xlabel('Targets (y_train)',size=18)
# plt.ylabel('Predictions (y_hat)',size=18)
# plt.show()

In [17]:
X_test = test[['CoordEncoded','EntryHeading','ExitHeading', 'Hour','Weekend']]
Y_test = model.predict(X_test)

In [18]:
tid = test['RowId'].astype(str)+'_'
tid

0                0_
1                1_
2                2_
3                3_
4                4_
             ...   
1921352    1921352_
1921353    1921353_
1921354    1921354_
1921355    1921355_
1921356    1921356_
Name: RowId, Length: 1921357, dtype: object

In [19]:
sub_model = pd.DataFrame()
sub_id = []
for i in tid:
    if i=="1920335_":
        break
    sub_id.append(i+'0')
    sub_id.append(i+'1')
    sub_id.append(i+'2')
    sub_id.append(i+'3')
    sub_id.append(i+'4')
    sub_id.append(i+'5')
sub_model['TargetId'] = sub_id
sub_model

,TargetId
0,0_0
1,0_1
2,0_2
3,0_3
4,0_4
...,...
11522005,1920334_1
11522006,1920334_2
11522007,1920334_3
11522008,1920334_4


In [20]:
Y_test = Y_test.flatten()
np.transpose(Y_test)
Y_test.shape,sub_model.shape

Y_test=Y_test[:11522010,]
sub_model['Target'] = Y_test
sub_model.to_csv('linear_reg.csv',index=False)